In [1]:
## Chap 2 modelling image classifier

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [5]:
def check_image(path):
  try:
    im = Image.open(path)
    return True
  except:
    return False

In [7]:
img_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

])

In [8]:
!pwd

/content


In [9]:
train_data_path = "/content/drive/MyDrive/Colab Notebooks/images/train"

train_data = torchvision.datasets.ImageFolder(root=train_data_path, 
                                              transform=img_transforms,
                                              is_valid_file=check_image
                                              )






In [10]:
val_data_path = "/content/drive/MyDrive/Colab Notebooks/images/val"

val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)



In [11]:
test_data_path = "/content/drive/MyDrive/Colab Notebooks/images/test"

test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=img_transforms,
                                             is_valid_file=check_image
                                             )



In [12]:
batch_size=64

**Loading the data**

In [14]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)


In [15]:
## create neural network  with 3 linear layers and ReLu activation
class SimpleNet(nn.Module):
  def __init__(self):
    super(SimpleNet, self).__init__()
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50, 2)

  def forward(self, x):
    x = x.view(-1, 12288)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [16]:
simplenet = SimpleNet()

In [17]:
## Create an Optimizer
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [18]:
### copy the model to the GPU if available
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")


simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [21]:
## training
# train model, 
# 1. copy batches to GPU if required, 
# 2. calculate losses,
# 3. optimize network,
# 4. validate each epoch

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
  for epoch in range(1, epochs+1):
    training_loss = 0.0
    valid_loss = 0.0
    model.train()
    for batch in train_loader:
      optimizer.zero_grad()
      inputs, targets = batch
      inputs = inputs.to(device)
      targets = targets.to(device)
      output = model(inputs)
      loss = loss_fn(output, targets)
      loss.backward()
      optimizer.step()
      training_loss += loss.data.item() * inputs.size(0)
    training_loss /= len(train_loader.dataset)

    model.eval()
    num_correct = 0
    num_examples = 0
    for batch in val_loader:
      inputs, targets = batch
      inputs = inputs.to(device)
      output = model(inputs)
      targets = targets.to(device)
      loss = loss_fn(output, targets)
      valid_loss += loss.data.item() * inputs.size(0)
      correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
      num_correct += torch.sum(correct).item()
      num_examples += correct.shape[0]
    valid_loss /= len(val_loader.dataset)

    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
                                                                                                valid_loss, num_correct / num_examples))



In [23]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), 
      train_data_loader, val_data_loader,
      epochs=5, device=device)



Epoch: 1, Training Loss: 2.22, Validation Loss: 4.47, accuracy = 0.30
Epoch: 2, Training Loss: 2.00, Validation Loss: 1.54, accuracy = 0.63
Epoch: 3, Training Loss: 0.89, Validation Loss: 1.13, accuracy = 0.50
Epoch: 4, Training Loss: 0.57, Validation Loss: 1.07, accuracy = 0.69
Epoch: 5, Training Loss: 0.36, Validation Loss: 1.03, accuracy = 0.57


In [24]:
## making predictions
labels = ['cat', 'fish']

img = Image.open("/content/drive/MyDrive/Colab Notebooks/images/val/fish/100_1422.JPG")
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction])



fish


In [31]:
## saving models
torch.save(simplenet, "/content/drive/MyDrive/Colab Notebooks/images/simplenet")
simplenet = torch.load("/content/drive/MyDrive/Colab Notebooks/images/simplenet")



In [32]:
torch.save(simplenet.state_dict(), "/content/drive/MyDrive/Colab Notebooks/simplenet")
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/simplenet")
simplenet.load_state_dict(simplenet_state_dict)



<All keys matched successfully>